In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing Initial Test Data

In [2]:
books_ctlg=pd.read_csv("BOOKSCATALOGUE.csv")
purchase_history=pd.read_csv("BOOKSPURCHHISTORY.csv")
visit_history=pd.read_csv("BOOKSVISITHISTORY.csv")
users_list=pd.read_csv("USERMASTER.csv")
train_data=pd.read_csv("BOOKSMASTERTRAIN.csv")

In [3]:
visit_history.head()

,UserID,BookID,SESSIONID,REFERRALID,TIMESTAMP,BOOKPREVIEWED,SUBSCRIBED,WeekofYear
0,637280,170387,276725249,3401168,2018-09-06 15:39:31,1,0,201836
1,656222,123033,277507996,3400002,2018-09-07 14:46:42,1,0,201836
2,660216,128265,276144807,3404417,2017-12-27 16:43:18,1,0,201752
3,654353,139601,276605171,3400002,2018-05-03 22:57:35,1,0,201818
4,614789,168842,275260040,3605387,2018-04-04 14:12:34,1,0,201814


In [4]:
users_list.head()

,UserID,SIGNUPDATE,DELETED,AGEGROUP,GENDER,STATE
0,600003,2018-09-07 01:29:02,NaN,28.0,FEMALE,NaN
1,600008,2017-11-02 20:33:20,NaN,43.0,MALE,NaN
2,600011,2017-11-21 02:03:30,NaN,35.0,MALE,NaN
3,600017,2018-02-13 12:21:22,2018-08-06 15:36:27,27.0,MALE,Massachusetts
4,600020,2017-11-13 22:29:13,NaN,53.0,MALE,Pennsylvania


# Mapping data and removing unncessary data(users_list)

In [5]:
users_list=users_list.drop(columns=['DELETED','STATE', 'SIGNUPDATE'], axis=1)
users_list.head()

,UserID,AGEGROUP,GENDER
0,600003,28.0,FEMALE
1,600008,43.0,MALE
2,600011,35.0,MALE
3,600017,27.0,MALE
4,600020,53.0,MALE


In [6]:
users_list['AGEGROUP']=np.where(users_list['AGEGROUP']>30,1,0)
users_list.head()

,UserID,AGEGROUP,GENDER
0,600003,0,FEMALE
1,600008,1,MALE
2,600011,1,MALE
3,600017,0,MALE
4,600020,1,MALE


In [7]:
users_list['GENDER']=users_list['GENDER'].map({'FEMALE':1,'MALE':0})
users_list.head()

,UserID,AGEGROUP,GENDER
0,600003,0,1
1,600008,1,0
2,600011,1,0
3,600017,0,0
4,600020,1,0


# Adding Purchased Books and Viewed Books for every user

In [8]:
books_purchased=[]
books_purchased_buffer=[]
users=[]
users=users_list['UserID']
for i in range(15000):
    books_purchased_buffer=purchase_history.loc[purchase_history['UserID'] == users[i]]
    books_purchased.append(books_purchased_buffer['BookID'].values)
    
books_purchased=[list(i) for i in books_purchased]
books_purchased

[[],
 [165498],
 [162488, 155019],
 [121582, 135346],
 [117564,
  105481,
  169328,
  167437,
  154892,
  114400,
  114845,
  121906,
  132243,
  109177,
  165478,
  117578],
 [120507, 175147, 170816, 136331, 126647, 143004, 129631],
 [],
 [170569],
 [129631, 111360, 126647, 170816, 145901, 138590, 138507, 156981],
 [],
 [153408, 153014, 117261, 138590, 142054, 129631, 143601, 156981, 161522],
 [130122],
 [150686, 172776, 103725, 146087],
 [151740,
  114400,
  141745,
  114400,
  130837,
  114400,
  121574,
  114400,
  131674,
  111351,
  148897],
 [119780],
 [],
 [125941, 101760],
 [],
 [],
 [143688, 170549],
 [162671],
 [127590],
 [108857],
 [128293],
 [],
 [158766],
 [164890, 128955, 123680, 140409, 128955, 125941],
 [159750],
 [163177],
 [],
 [],
 [],
 [],
 [148175],
 [121582, 175147, 138590, 147044, 173346, 122549, 129631, 121906, 176995],
 [106808],
 [164551, 146658, 108923],
 [104241],
 [141194,
  161020,
  166578,
  131571,
  154562,
  102688,
  110860,
  128592,
  105383,
  15

In [9]:
users_list['books_bought']=books_purchased#adding books bought by every user to user list
users_list['books_bought']#=users_list['books_bought'].str[0]
users_list

,UserID,AGEGROUP,GENDER,books_bought
0,600003,0,1,[]
1,600008,1,0,[165498]
2,600011,1,0,"[162488, 155019]"
3,600017,0,0,"[121582, 135346]"
4,600020,1,0,"[117564, 105481, 169328, 167437, 154892, 11440..."
5,600028,1,1,"[120507, 175147, 170816, 136331, 126647, 14300..."
6,600031,1,0,[]
7,600038,1,1,[170569]
8,600043,1,1,"[129631, 111360, 126647, 170816, 145901, 13859..."
9,600046,1,1,[]


In [10]:
books_visited=[]
books_visited_buffer=[]
for i in range(15000):
    books_visited_buffer=visit_history.loc[visit_history['UserID'] == users[i]]
    books_visited.append(books_visited_buffer['BookID'].values)
    
books_visited=[list(i) for i in books_visited]
books_visited

[[109745],
 [170179,
  148897,
  126430,
  162822,
  123350,
  173023,
  118789,
  165498,
  122871,
  163723,
  163430,
  156587,
  154530,
  124400,
  147044,
  100464,
  163723,
  165498],
 [155019, 155019, 155019, 177754],
 [121582, 135346, 135346, 135346, 135346],
 [115225,
  169328,
  121906,
  114845,
  171884,
  121906,
  108927,
  154059,
  121956,
  130472,
  132243,
  140330,
  117564,
  154326,
  150975,
  128406,
  171884,
  167437,
  128406,
  117564,
  165478,
  102624,
  125509,
  169328,
  119374,
  130928,
  117564,
  162785,
  105481,
  122740,
  132243,
  149377,
  107108,
  114400,
  125509,
  117578,
  117564,
  174797,
  156752,
  147217,
  100409,
  114845,
  153927,
  167437,
  176625,
  144522,
  102825,
  114400,
  100887,
  145919,
  150865,
  121906,
  165478,
  150242,
  176742,
  169328,
  161536,
  134871,
  102825,
  105114,
  179571,
  121906,
  108927,
  179663,
  147217,
  114845,
  117578,
  120202,
  119374,
  132238,
  139599,
  109177,
  165478,


In [11]:
users_list['books_visited']=books_visited #adding books visited by every user to user list
users_list.head()
users_list['books_relevant']=users_list['books_visited']+users_list['books_bought']#adding books_bought and books_visited
users_list=users_list.drop(columns=['books_bought','books_visited'], axis=1)
users_list

,UserID,AGEGROUP,GENDER,books_relevant
0,600003,0,1,[109745]
1,600008,1,0,"[170179, 148897, 126430, 162822, 123350, 17302..."
2,600011,1,0,"[155019, 155019, 155019, 177754, 162488, 155019]"
3,600017,0,0,"[121582, 135346, 135346, 135346, 135346, 12158..."
4,600020,1,0,"[115225, 169328, 121906, 114845, 171884, 12190..."
5,600028,1,1,"[175147, 126647, 136331, 138507, 117756, 15030..."
6,600031,1,0,"[116882, 100804, 121582, 118679, 173406, 16547..."
7,600038,1,1,"[170569, 170569, 132893, 170569, 170569, 17056..."
8,600043,1,1,"[123784, 177211, 147044, 177061, 163177, 14360..."
9,600046,1,1,[165088]


# Adding Genres, book type, popularity, user rating, coverpage

In [12]:
Genre=[]
Genre_buffer=[]
books_bought=users_list['books_relevant'].values
for i in range(15000):
    temp=[]
    temp=books_bought[i]
    length=len(temp)
    temp2=[]
    #print(length)
    
    for j in range(length):
        Genre_buffer=train_data.loc[train_data['BookID'] == temp[j]]
        temp2.append(Genre_buffer['GENRE'].values)
        Genre_temp=[list(i) for i in temp2]

    Genre.append(Genre)

In [ ]:
users_list['Genres_relevant']=Genre
users_list.head()